In [36]:
import pandas as pd
import numpy as np
import json

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
UNRELEASED_SELECT = '//select[@class="font-medium rounded-md text-blazer-950 text-right pb-0.5"]'
CLOSE_SETTINGS_MENU_ICON = '//div[@class="absolute right-3 cursor-pointer p-4"]'
CHAR_SELECT_ICON = '//a[@href="/en/archive/avatar"]'
CHAR_GRID = '//div[@class="flex flex-row flex-wrap justify-center gap-6 mt-2"]'
CHAR_NAME = './/div[@class="text-2xl font-bold text-white xl:text-3xl"]'

class StatMiner:
    def __init__(self, target_url):
        self.target_url = target_url
        self.driver = None
    
    def create_driver(self):
        self.driver = webdriver.Chrome()
        self.driver.get(self.target_url)
        self.driver.maximize_window()

    def close_menu(self):
        # close the settings menu
        settings_title = self.driver.find_element(By.XPATH, '//div[@name="title"]')
        x_button = settings_title.find_element(By.XPATH, CLOSE_SETTINGS_MENU_ICON)
        x_button.click()

    def get_chars(self):
        # find the div which contains all char links
        char_grid = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, CHAR_GRID))
        )
        all_characters = char_grid.find_elements(By.TAG_NAME, 'a')

        # loop
        df_list = []
        name_list = []
        for i in range(len(all_characters)):
            char_grid = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, CHAR_GRID))
            )
            all_characters = char_grid.find_elements(By.TAG_NAME, 'a')
            all_characters[i].click()

            # find character name
            char_name_div = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'Avatar Name'))
            )
            char_name = char_name_div.find_element(By.XPATH, CHAR_NAME).get_attribute('innerText')
            name_list.append(char_name)
            print(char_name)
            
            # find the slider button to convert to table
            slider_button = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//button[text()="Slider"]'))
            )
            slider_button.click()

            # get a dataframe from the table
            stat_table = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'STAT TABLE'))
            )
            stat_table_html = stat_table.get_attribute('outerHTML')
            char_df = pd.read_html(stat_table_html)[0]
            df_list.append(char_df)

            self.driver.back()
        return df_list, name_list

In [3]:
YATTA_URL = 'https://hsr.yatta.top/en/archive/avatar'
scraper = StatMiner(YATTA_URL)
scraper.create_driver()
scraper.close_menu()
df_list, name_list = scraper.get_chars()

Misha
Sparkle
Black Swan
Dr. Ratio
Xueyi
Ruan Mei
Hanya
Argenti
Huohuo
Topaz & Numby
Guinaifen
Jingliu
Lynx
Fu Xuan
Dan Heng • Imbibitor Lunae
Kafka
Luka
Blade
Luocha
Yukong
Silver Wolf
Jing Yuan
March 7th
Dan Heng
Himeko
Welt
Arlan
Asta
Herta
Bronya
Seele
Serval
Gepard
Natasha
Pela
Clara
Sampo
Hook
Qingque
Tingyun
Sushang
Yanqing
Bailu
Trailblazer
Trailblazer
Trailblazer
Trailblazer


In [75]:
TRAILBLAZER_ORDER = [
    'Caelus - Destruction',
    'Stelle - Destruction',
    'Caelus - Preservation',
    'Stelle - Preservation'
]
dfs = df_list.copy()
names = name_list.copy()

def convert_to_json(df):
    char_name = df['Name'][0]
    df = df.T
    df.columns = df.iloc[1, :]
    df = df.iloc[2:, :]
    df = df.fillna(method='ffill')
    df['CRIT Rate'] = df['CRIT Rate'].replace('[%.]', '', regex=True).astype(float)/1000
    df['CRIT DMG'] = df['CRIT DMG'].replace('[%.]', '', regex=True).astype(float)/1000
    for col in df.columns:
        df[col] = df[col].astype(float)
    df_json = json.loads(df.to_json())
    return_json = {char_name: df_json}
    return return_json

character_stats = []
for df in dfs:
    character_stats.append(convert_to_json(df))

with open('../character_logic/data/hsr_char_stats.json', 'w') as f:
    json.dump(character_stats, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: '../data/hsr_char_stats.json'

In [8]:
'''
dfs = df_list.copy()
names = name_list.copy()

for i in range(len(dfs)):
    dfs[i]['LVL'] = dfs[i]['LVL'].fillna('Aggro')
    dfs[i].insert(0, 'Name', [names[i], *[np.nan for i in range(len(dfs[i]) - 1)]])

df = pd.concat(dfs)

numeric_cols = [col for col in df.columns if any(char.isdigit() for char in col)]

def convert_values(value):
    value_str = str(value)
    if '%' in value_str:
        return float(value_str.strip('%'))/100
    else:
        return float(value_str)

for col in numeric_cols:
    df[col] = df[col].apply(convert_values)

df.to_json('hsr_char_stats.json')'''

ValueError: DataFrame index must be unique for orient='columns'.